In [1]:
import pandas as pd
import numpy as np

In [3]:
file_path = "data/sample_sales_data.csv"

In [4]:
def top_n_products(n, operation):

    OPERATION = operation.lower()
    df_data = pd.read_csv(file_path)

    df_temp = df_data[df_data['OPERATION_NAME'].str.lower().str.contains(OPERATION)]\
                    .groupby(['PRODUCT_ID','PRODUCT_NAME'])['UNITS_SOLD'].sum().reset_index()
    df_top_n = df_temp.sort_values("UNITS_SOLD", ascending = False).head(n)

    return df_top_n

In [5]:
top_n_products(3, 'DALLAS')

,PRODUCT_ID,PRODUCT_NAME,UNITS_SOLD
4046,PMA-93040,Niagara Purified Drinking Water (16.9oz),875040
2517,PMA-40579,Ozarka Water (16.9oz),816552
3055,PMA-5318,Pure Life Purified Water (16.9oz),676080


In [9]:
import pandas as pd

def prophet_forecast_next_6_months(y: pd.Series, include_history: bool = True) -> pd.DataFrame:
    """
    Forecast a univariate time series for the next 6 months using Prophet.

    Parameters
    ----------
    y : pd.Series
        Time series values with a DatetimeIndex.
    include_history : bool, default True
        Whether to include historical dates in the returned forecast.

    Returns
    -------
    pd.DataFrame
        Prophet forecast dataframe including columns like:
        ["ds", "yhat", "yhat_lower", "yhat_upper", ...]
        For convenience, will also include a datetime index on "ds".
    """
    # Import Prophet with compatibility across package names
    try:
        from prophet import Prophet
    except ImportError:  # older installs
        from fbprophet import Prophet  # type: ignore

    if not isinstance(y, pd.Series):
        raise TypeError("Input must be a pandas Series.")
    if not isinstance(y.index, pd.DatetimeIndex):
        raise TypeError("Series must have a DatetimeIndex.")

    # Ensure sorted and clean
    y = y.sort_index()
    df = y.rename("y").to_frame()
    df = df[~df["y"].isna()].reset_index().rename(columns={df.index.name or "index": "ds"})
    df["ds"] = pd.to_datetime(df["ds"])

    if df.empty:
        raise ValueError("Series has no non-NA data points.")

    # Fit model
    m = Prophet()
    m.fit(df)

    # Future: next 6 months from last observed date
    last_date = df["ds"].max()
    future_end = last_date + pd.DateOffset(months=6)
    future_dates = pd.date_range(start=last_date, end=future_end, freq='MS')

    # Prophet wants a DataFrame with a 'ds' column
    if include_history:
        future = pd.DataFrame({"ds": pd.concat([df["ds"], pd.Series(future_dates)], ignore_index=True).drop_duplicates().sort_values()})
    else:
        future = pd.DataFrame({"ds": future_dates})

    forecast = m.predict(future)

    # Helpful index
    forecast = forecast.copy()
    forecast["ds"] = pd.to_datetime(forecast["ds"])
    forecast = forecast.set_index("ds")

    # Return the full Prophet forecast (user can select cols)
    return round(forecast['yhat'])


In [10]:
def forecast(operation):

    OPERATION = operation.lower()
    df_data = pd.read_csv(file_path)

    df_data_temp = df_data[df_data['OPERATION_NAME'].str.lower().str.contains(OPERATION)]\
        .groupby(['CALENDAR_YEAR','CALENDAR_MONTH'])\
        ['UNITS_SOLD'].sum().reset_index()

    df_data_temp['Date'] = (
        df_data_temp['CALENDAR_YEAR'].astype(str)
        + '-'
        + df_data_temp['CALENDAR_MONTH'].astype(str).str.zfill(2)
        + '-01'
    )
    df_data_temp['Date'] = pd.to_datetime(df_data_temp['Date'])
    df_data_temp = df_data_temp.sort_values('Date')
    input_series = df_data_temp.set_index('Date')['UNITS_SOLD']

    df_forecast = prophet_forecast_next_6_months(input_series)


    return df_forecast

In [11]:
forecast('Dallas')

/opt/anaconda3/envs/portfolio-ai-backend/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
15:51:41 - cmdstanpy - INFO - Chain [1] start processing
15:51:42 - cmdstanpy - INFO - Chain [1] done processing


ds
2024-09-01     920897.0
2024-10-01     965840.0
2024-11-01    1012280.0
2024-12-01    1057223.0
2025-01-01    1103664.0
2025-02-01    1150105.0
2025-03-01    1192051.0
2025-04-01    1238492.0
2025-05-01    1283434.0
2025-06-01    1329875.0
2025-07-01    1374818.0
2025-08-01    1421259.0
2025-09-01    1467699.0
2025-10-01    1512642.0
2025-11-01    1559083.0
2025-12-01    1604025.0
2026-01-01    1650466.0
2026-02-01    1696907.0
2026-03-01    1738853.0
Name: yhat, dtype: float64

In [ ]:
##TODO: 
# 1. MCP server/client that you have developed where are you going to host it? 
# 2. Google drive(external) => google sdk -> how to connect it with MCP?
# 3. High level architecture (mcp server-client and connection)
# 4. Cloud based hosting